In [21]:
import pandas as pd
import re
import spacy
from tqdm import tqdm

tqdm.pandas()

In [24]:
en_train_path = "../data/processed/processed_CT24_checkworthy_english/sentence_split_processed_train.tsv"  # noqa: E501
du_train_path = "../data/processed/processed_CT24_checkworthy_dutch/sentence_split_processed_dutch_train.tsv"  # noqa: E501
es_train_path = "../data/processed/processed_CT24_checkworthy_spanish/sentence_split_processed_spanish_train.tsv"  # noqa: E501
ar_train_path = "../data/processed/processed_CT24_checkworthy_arabic/sentence_split_processed_arabic_train.tsv"  # noqa: E501

en_train = pd.read_csv(en_train_path, sep="\t")
du_train = pd.read_csv(du_train_path, sep="\t")
es_train = pd.read_csv(es_train_path, sep="\t")
ar_train = pd.read_csv(ar_train_path, sep="\t")

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/processed_CT24_checkworthy_arabic/sentence_split_arabic_processed_train.tsv'

In [ ]:
import re


def correct_tags(text):
    pattern = r"(<s>)+|(<\/s>)+"

    def replace_func(match):
        tag = match.group(0)
        if "<s>" in tag:
            return "<s>"
        else:
            return "</s>"

    corrected_text = re.sub(pattern, replace_func, text)
    return corrected_text

In [ ]:
# https://spacy.io/models
# python -m spacy download en_core_web_trf
# python -m spacy download nl_core_news_lg
# python -m spacy download es_dep_news_trf
# python -m spacy download xx_sent_ud_sm

spacy_models = {
    "en": "en_core_web_sm",
    "nl": "nl_core_news_lg",
    "es": "es_core_news_sm",
    "ar": "xx_sent_ud_sm",
}

nlp_models = {lang: spacy.load(model) for lang, model in spacy_models.items()}

for dataset, dataset_path, lang in [
    (en_train, en_train_path, "en"),
    (du_train, du_train_path, "nl"),
    (es_train, es_train_path, "es"),
    (ar_train, ar_train_path, "ar"),
]:

    def add_stop_token(text: str) -> str:
        # if lang not in nlp_models:
        #     raise ValueError("Ugyldig språk")

        # nlp = nlp_models[lang]
        # text = text.replace("#", " ")
        # doc = nlp(text)
        # sentences = []

        # for sent in doc.sents:
        #     if re.search(r"\b(?:[A-Z]\.)+\b", sent.text):
        #         next_sent = doc[sent.end].text if sent.end < len(doc) else ""
        #         combined_sent = sent.text + next_sent
        #         sentences.append(f"<s> {combined_sent.strip()} </s>")
        #     else:
        #         sentences.append(f"<s> {sent.text.strip()} </s>")

        # sentences = " ".join(sentences)
        # sentences = correct_tags(sentences)

        # sentences = re.sub(r"<s>(.*?<s>.*?</s>.*?)</s>", r"\1", sentences)
        # sentences = re.sub(r"<s>(.*?</s>.*?)<s>", r"\1", sentences)
        text = text.replace("\n", " ")
        text = text.replace("\t", " ")
        text = text.replace("<s>", "")
        text = text.replace("</s>", "")
        text = re.sub(r" +", " ", text)
        return text

    def correct_tags(text: str) -> str:
        pattern = r"(<s>)+|(<\/s>)+"

        def replace_func(match):
            return "<s>" if "<s>" in match.group(0) else "</s>"

        corrected_text = re.sub(pattern, replace_func, text)
        return corrected_text

    dataset["tweet_text"] = dataset["tweet_text"].progress_apply(
        lambda x: x.replace("<s>", "").replace("</s>", "")
    )
    dataset["tweet_text"] = dataset["tweet_text"].progress_apply(add_stop_token)

    display(dataset)

    dataset.to_csv(dataset_path, sep="\t", index=False)

100%|██████████| 10000/10000 [00:00<00:00, 197929.49it/s]


,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,12818,I think we have to reduce that level of tensio...,No,22,NaN,[],21-30,0,0.0,4
1,7542,Electricity and brick and mortar were always h...,No,25,NaN,[],21-30,0,0.0,0
2,12291,"Without a doubt, we must appeal to unity, but ...",No,14,NaN,[],11-20,0,0.0,3
3,1326,"@<USER> : ""This government is showing that it ...",Yes,46,#IreneMonteroEnJELO,['@IreneMontero:'],41-50,1,0.0,3
4,10217,It's no surprise that it seems like a market l...,No,23,NaN,[],21-30,0,0.0,4
...,...,...,...,...,...,...,...,...,...,...
9995,16099,The Government of Spain has a firm commitment ...,No,31,NaN,[],31-40,0,0.0,1
9996,10042,If this oversight serves to make us all aware ...,No,13,NaN,[],11-20,0,0.0,4
9997,2576,The cruel and criminal blow of the #ViolenciaD...,No,46,#ViolenciaDeGénero #NiUnaMenos,[],41-50,2,0.0,1
9998,8661,Judges cannot elect judges. Bolaños passes thr...,Yes,42,NaN,[],41-50,0,0.0,1


100%|██████████| 10000/10000 [00:00<00:00, 199814.40it/s]


,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,6761,In juli 2018<USER> @ gaf een levendige toespra...,No,38,#CreemosElCambio,"['@pablocasado_', '@populares']",31-40,1,0.0,3
1,2027,"De voorzitter van Renfe, Isaías Taboas, overha...",No,25,"#Avlo,","['@mitmagob,', '@abalosmeco']",21-30,1,0.0,0
2,2977,"@<USER> nl : ""Bildu Sortu en Otegi zijn nu de ...",No,19,#Euskadi,"['@andoniortuzar,', '@Herri_Irratia:']",11-20,1,0.0,3
3,18093,En als deze mensen gelukkig zijn en je vooral ...,No,22,NaN,[],21-30,0,0.0,4
4,6284,Interview met Joan Torres: Barcelona kan zijn ...,No,19,NaN,['@BCNmetropoli'],11-20,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
9995,6341,Het zijn lange maanden van onderhandelingen ge...,No,39,NaN,[],31-40,0,0.0,0
9996,10244,Als Fernández Díaz Cospedal of andere leden va...,No,20,NaN,[],11-20,0,0.0,3
9997,2617,De moord op Ana Orantes schokte ons land. Hun ...,No,39,#ViolenciaMachista,['@nytimes'],31-40,1,0.0,1
9998,2084,We willen onze @ bedanken<USER> voor haar serv...,No,29,#PoliciaMunicipalDeMadrid #SanJuan2020,['@policiademadrid'],21-30,2,0.0,1


100%|██████████| 10000/10000 [00:00<00:00, 223235.46it/s]


,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,197,En los últimos cuatro años se han perdido medi...,Yes,74,NaN,NaN,71-80,0,0.0,0
1,11717,"Y, por lo tanto, solo los niños estaban siendo...",No,63,NaN,NaN,61-70,0,0.0,0
2,11071,"No puedes liderar una alianza si dices, ya sab...",No,71,NaN,NaN,71-80,0,0.0,0
3,2136,"Dicen, ¿para qué necesitamos esto?",No,37,NaN,NaN,31-40,0,0.0,0
4,7768,Creo que si vas a tener desgravaciones fiscale...,No,65,NaN,NaN,61-70,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
9995,11965,Pero conducen en direcciones muy diferentes.,No,43,NaN,NaN,41-50,0,0.0,0
9996,19920,Y tengo que defender el honor de mi estado.,No,43,NaN,NaN,41-50,0,0.0,0
9997,2480,Y creo que tenemos que tener una oleada de int...,No,107,NaN,NaN,100+,0,0.0,0
9998,22443,Siguen siendo el número uno hoy en día.,Yes,31,NaN,NaN,31-40,0,0.0,0


100%|██████████| 10000/10000 [00:00<00:00, 216949.38it/s]


,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,1539,"كارمن كالفو: ""ما الذي تنوي فعله يا سيد كاسادو؟...",Yes,34,#SesionDeControl,[],31-40,1,0.0,2
1,3977,كل عام زاد عدد الحضور والعارضين.لكنه ليس مجرد ...,Yes,39,NaN,"['@govern,', '@MWCapital,', '@bcn_ajuntament']",31-40,0,0.0,0
2,18634,حسنا ، رؤية الإخفاقات التي واجهناها أو الصعوبا...,No,11,NaN,[],11-20,0,0.0,4
3,12741,حسنا ، أعتقد أنني أخبرتك بذلك من قبل.,No,8,NaN,[],0-10,0,0.0,4
4,5047,هذه الحكومة هي حظيرة دجاج طوال اليوم.الوزيرة ب...,No,42,NaN,['@GPPopular'],41-50,0,0.0,3
...,...,...,...,...,...,...,...,...,...,...
9995,8611,كل شيء إلى @<USER> موقع انظر مظهر (تم نشره بال...,No,15,NaN,"['@Congreso_Es', '@fjconpe']",11-20,0,0.0,3
9996,11919,لماذا نرى الأسود على الأبيض يصور 50 عاما من تا...,No,18,NaN,[],11-20,0,0.0,4
9997,13221,ثم ، بعد ذلك بقليل ،,No,6,NaN,[],0-10,0,0.0,4
9998,1488,سأخبرك حكاية تكشف جيدا كيف يبدو رجال ونساء الح...,No,45,NaN,[],41-50,0,0.0,1
